In [1]:
import scanpy as sc

In [5]:
adata = sc.read_h5ad("/SGRNJ06/randd/USER/zhouyiqi/work/analysis/20250721-vdj-crosstalk/public-10x/c1754_fig_1b_cat.h5ad")

In [48]:
Counter(adata.obs['annot_full'])

Counter({'T cells': 65276,
         'Mononuclear phagocytes': 4523,
         'Alveolar type II cells': 2330,
         'Doublet': 1136,
         'Goblet cells': 1090,
         'B cells': 204,
         'Plasma cells': 195,
         'Endothelial cells': 137,
         'Erythrocytes': 33,
         'Plasmacytoid dendritic cells': 19,
         'Mast cells': 16})

In [12]:
import pandas as pd
tcr_barcodes = pd.read_csv('/SGRNJ06/randd/USER/zhouyiqi/work/analysis/20250721-vdj-crosstalk/public-10x/GSM4952981_P61_Blood_filtered_contig_annotations.csv').barcode.drop_duplicates()
tcr_barcodes

0        AAACCTGAGAGCTATA-1
9        AAACCTGAGCAGACTG-1
12       AAACCTGCACCAGTTA-1
16       AAACCTGCACGGCGTT-1
21       AAACCTGTCCAAACAC-1
                ...        
15287    TTTGTCACACATGTGT-1
15292    TTTGTCACAGTTCCCT-1
15295    TTTGTCACATCACGAT-1
15302    TTTGTCACATTGGTAC-1
15311    TTTGTCATCAACGGGA-1
Name: barcode, Length: 3352, dtype: object

In [31]:
sample = 'P61_Blood'
tcr_barcodes = pd.read_csv(f'/SGRNJ06/randd/USER/zhouyiqi/work/analysis/20250721-vdj-crosstalk/public-10x/GSM4952981_{sample}_filtered_contig_annotations.csv').barcode.drop_duplicates()
p61 = adata.obs[adata.obs['Sample ID'] == sample].copy()

# 去掉前缀，只保留 barcode
p61['barcode'] = p61.index.str.split('_').str[-1]

# 标记是否组装出 TCR
tcr_set = set(tcr_barcodes)
print('sample:', sample)
print('total cell: ', len(p61['barcode']))
print('total T cell: ', sum(p61['annot_full'] == 'T cells'))
p61['has_TCR'] = p61['barcode'].isin(tcr_set)

# 只看有 TCR 的细胞
p61_has_tcr = p61[p61['has_TCR']]

# 标记是否为 T 细胞
p61_has_tcr.loc[:, 'is_T_cell'] = p61_has_tcr['annot_full'] == 'T cells'

# 统计数量
count_series = p61_has_tcr['is_T_cell'].value_counts()
total = count_series.sum()
t_count = count_series.get(True, 0)
non_t_count = count_series.get(False, 0)

# 输出格式：数量(百分比%)
def format_out(count):
    return f"{count} ({count/total*100:.2f}%)" if total > 0 else "0 (0.00%)"

print("Cells with assembled TCR:")
print(f"- T cells: {format_out(t_count)}")
print(f"- Non-T cells: {format_out(non_t_count)}")

sample: P61_Blood
total cell:  5749
total T cell:  5049
Cells with assembled TCR:
- T cells: 3213 (99.11%)
- Non-T cells: 29 (0.89%)


/tmp/ipykernel_1386/1730614052.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p61_has_tcr.loc[:, 'is_T_cell'] = p61_has_tcr['annot_full'] == 'T cells'


In [36]:
import glob
sample = 'P61_Tumor'
csv_file = glob.glob(f'/SGRNJ06/randd/USER/zhouyiqi/work/analysis/20250721-vdj-crosstalk/public-10x/*{sample}*_annotations.csv')[0]
tcr_barcodes = pd.read_csv(csv_file).barcode.drop_duplicates()
p61 = adata.obs[adata.obs['Sample ID'] == sample].copy()

# 去掉前缀，只保留 barcode
p61['barcode'] = p61.index.str.split('_').str[-1]

# 标记是否组装出 TCR
tcr_set = set(tcr_barcodes)
print('sample:', sample)
print('total cell: ', len(p61['barcode']))
print('total T cell: ', sum(p61['annot_full'] == 'T cells'))
p61['has_TCR'] = p61['barcode'].isin(tcr_set)

# 只看有 TCR 的细胞
p61_has_tcr = p61[p61['has_TCR']]

# 标记是否为 T 细胞
p61_has_tcr.loc[:, 'is_T_cell'] = p61_has_tcr['annot_full'] == 'T cells'

# 统计数量
count_series = p61_has_tcr['is_T_cell'].value_counts()
total = count_series.sum()
t_count = count_series.get(True, 0)
non_t_count = count_series.get(False, 0)

# 输出格式：数量(百分比%)
def format_out(count):
    return f"{count} ({count/total*100:.2f}%)" if total > 0 else "0 (0.00%)"

print("Cells with assembled TCR:")
print(f"- T cells: {format_out(t_count)}")
print(f"- Non-T cells: {format_out(non_t_count)}")

sample: P61_Tumor
total cell:  5208
total T cell:  3740
Cells with assembled TCR:
- T cells: 2354 (97.96%)
- Non-T cells: 49 (2.04%)


/tmp/ipykernel_1386/3979504313.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p61_has_tcr.loc[:, 'is_T_cell'] = p61_has_tcr['annot_full'] == 'T cells'


In [42]:
from collections import Counter
Counter(adata.obs['orig.ident'])

Counter({'P60_Blood': 10661,
         'P58_Blood': 10500,
         'P60_Tumor': 10366,
         'P61_Blood': 5749,
         'P42_Tumor': 5235,
         'P61_Tumor': 5208,
         'P47_Tumor': 4752,
         'P57_Tumor': 3850,
         'P43_Tumor': 2950,
         'P60_Juxta': 2315,
         'P55_Tumor': 2190,
         'P46_Tumor': 2173,
         'P35_Tumor': 2075,
         'P61_Juxta': 1921,
         'P58_Tumor': 1690,
         'P55_Blood': 1689,
         'P34_Tumor': 1635})

In [43]:
import pandas as pd
import glob

csv_files = glob.glob('/SGRNJ06/randd/USER/zhouyiqi/work/analysis/20250721-vdj-crosstalk/public-10x/*_filtered_contig_annotations.csv')

summary_list = []

for csv_file in csv_files:
    sample = csv_file.split('/')[-1].split('_')[1] + '_' + csv_file.split('/')[-1].split('_')[2]
    if sample == 'P57_Blood':
        continue

    tcr_barcodes = pd.read_csv(csv_file).barcode.drop_duplicates()
    tcr_set = set(tcr_barcodes)

    p = adata.obs[adata.obs['Sample ID'] == sample].copy()
    p['barcode'] = p.index.str.split('_').str[-1]

    total_cells = len(p)
    total_t_cells = sum(p['annot_full'] == 'T cells')

    p['has_TCR'] = p['barcode'].isin(tcr_set)
    p_has_tcr = p[p['has_TCR']].copy()
    p_has_tcr['is_T_cell'] = p_has_tcr['annot_full'] == 'T cells'

    count_series = p_has_tcr['is_T_cell'].value_counts()
    tcr_in_t_cells = count_series.get(True, 0)
    tcr_in_non_t_cells = count_series.get(False, 0)

    total_tcr_cells = tcr_in_t_cells + tcr_in_non_t_cells
    if total_tcr_cells == 0:
        tcr_in_t_cells_pct = 0
        tcr_in_non_t_cells_pct = 0
    else:
        tcr_in_t_cells_pct = tcr_in_t_cells / total_tcr_cells * 100
        tcr_in_non_t_cells_pct = tcr_in_non_t_cells / total_tcr_cells * 100

    summary_list.append({
        'Sample': sample,
        'Total Cells': total_cells,
        'Total T Cells': total_t_cells,
        'TCR in T cells': f"{tcr_in_t_cells} ({tcr_in_t_cells_pct:.2f}%)",
        'TCR in non-T cells': f"{tcr_in_non_t_cells} ({tcr_in_non_t_cells_pct:.2f}%)"
    })

summary_df = pd.DataFrame(summary_list)
print(summary_df)


       Sample  Total Cells  Total T Cells  TCR in T cells TCR in non-T cells
0   P47_Tumor         4752           4227   3171 (98.54%)         47 (1.46%)
1   P60_Tumor        10366           9678   8200 (98.83%)         97 (1.17%)
2   P60_Juxta         2315           1917   1451 (98.37%)         24 (1.63%)
3   P60_Blood        10661          10398  10071 (98.28%)        176 (1.72%)
4   P58_Blood        10500          10341   6782 (98.38%)        112 (1.62%)
5   P61_Blood         5749           5049   3213 (99.11%)         29 (0.89%)
6   P55_Tumor         2190           2137   1474 (99.93%)          1 (0.07%)
7   P61_Tumor         5208           3740   2354 (97.96%)         49 (2.04%)
8   P61_Juxta         1921            875    506 (86.79%)        77 (13.21%)
9   P58_Tumor         1690           1535    834 (95.31%)         41 (4.69%)
10  P57_Tumor         3850           1925   1701 (97.98%)         35 (2.02%)


In [47]:
import pandas as pd
import glob

csv_files = glob.glob('/SGRNJ06/randd/USER/zhouyiqi/work/analysis/20250721-vdj-crosstalk/public-10x/*_filtered_contig_annotations.csv')

summary_list = []

for csv_file in csv_files:
    sample = csv_file.split('/')[-1].split('_')[1] + '_' + csv_file.split('/')[-1].split('_')[2]
    if sample == 'P57_Blood':
        continue
    tcr_barcodes = pd.read_csv(csv_file).barcode.drop_duplicates()
    tcr_set = set(tcr_barcodes)

    p = adata.obs[adata.obs['Sample ID'] == sample].copy()
    p['barcode'] = p.index.str.split('_').str[-1]

    total_cells = len(p)
    total_t_cells = sum(p['annot_full'] == 'T cells')
    total_non_t_cells = total_cells - total_t_cells

    p['has_TCR'] = p['barcode'].isin(tcr_set)
    p_has_tcr = p[p['has_TCR']].copy()
    p_has_tcr['is_T_cell'] = p_has_tcr['annot_full'] == 'T cells'

    count_series = p_has_tcr['is_T_cell'].value_counts()
    tcr_in_t_cells = count_series.get(True, 0)
    tcr_in_non_t_cells = count_series.get(False, 0)

    total_tcr_cells = tcr_in_t_cells + tcr_in_non_t_cells
    if total_tcr_cells == 0:
        tcr_in_t_cells_pct = 0
        tcr_in_non_t_cells_pct = 0
    else:
        tcr_in_t_cells_pct = tcr_in_t_cells / total_tcr_cells * 100
        tcr_in_non_t_cells_pct = tcr_in_non_t_cells / total_tcr_cells * 100

    t_cell_with_tcr_pct = (tcr_in_t_cells / total_t_cells * 100) if total_t_cells > 0 else 0
    non_t_cell_with_tcr_pct = (tcr_in_non_t_cells / total_non_t_cells * 100) if total_non_t_cells > 0 else 0

    def format_out(count, pct):
        return f"{count} ({pct:.2f}%)"

    summary_list.append({
        'Sample': sample,
        'Total Cells': total_cells,
        'Total T Cells': total_t_cells,
        'T cell with TCR': format_out(tcr_in_t_cells, t_cell_with_tcr_pct),
        'Non-T cell with TCR': format_out(tcr_in_non_t_cells, non_t_cell_with_tcr_pct),
        'TCR in T cells': format_out(tcr_in_t_cells, tcr_in_t_cells_pct),
        'TCR in non-T cells': format_out(tcr_in_non_t_cells, tcr_in_non_t_cells_pct)
    })

summary_df = pd.DataFrame(summary_list)
summary_df


,Sample,Total Cells,Total T Cells,T cell with TCR,Non-T cell with TCR,TCR in T cells,TCR in non-T cells
0,P47_Tumor,4752,4227,3171 (75.02%),47 (8.95%),3171 (98.54%),47 (1.46%)
1,P60_Tumor,10366,9678,8200 (84.73%),97 (14.10%),8200 (98.83%),97 (1.17%)
2,P60_Juxta,2315,1917,1451 (75.69%),24 (6.03%),1451 (98.37%),24 (1.63%)
3,P60_Blood,10661,10398,10071 (96.86%),176 (66.92%),10071 (98.28%),176 (1.72%)
4,P58_Blood,10500,10341,6782 (65.58%),112 (70.44%),6782 (98.38%),112 (1.62%)
5,P61_Blood,5749,5049,3213 (63.64%),29 (4.14%),3213 (99.11%),29 (0.89%)
6,P55_Tumor,2190,2137,1474 (68.98%),1 (1.89%),1474 (99.93%),1 (0.07%)
7,P61_Tumor,5208,3740,2354 (62.94%),49 (3.34%),2354 (97.96%),49 (2.04%)
8,P61_Juxta,1921,875,506 (57.83%),77 (7.36%),506 (86.79%),77 (13.21%)
9,P58_Tumor,1690,1535,834 (54.33%),41 (26.45%),834 (95.31%),41 (4.69%)


In [45]:
summary_df

,Sample,Total Cells,Total T Cells,T cell with TCR,Non-T cell with TCR,TCR in T cells,TCR in non-T cells
0,P47_Tumor,4752,4227,3171 (75.02%),47 (8.95%),3171 (98.54%),47 (1.46%)
1,P60_Tumor,10366,9678,8200 (84.73%),97 (14.10%),8200 (98.83%),97 (1.17%)
2,P60_Juxta,2315,1917,1451 (75.69%),24 (6.03%),1451 (98.37%),24 (1.63%)
3,P57_Blood,0,0,0 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%)
4,P60_Blood,10661,10398,10071 (96.86%),176 (66.92%),10071 (98.28%),176 (1.72%)
5,P58_Blood,10500,10341,6782 (65.58%),112 (70.44%),6782 (98.38%),112 (1.62%)
6,P61_Blood,5749,5049,3213 (63.64%),29 (4.14%),3213 (99.11%),29 (0.89%)
7,P55_Tumor,2190,2137,1474 (68.98%),1 (1.89%),1474 (99.93%),1 (0.07%)
8,P61_Tumor,5208,3740,2354 (62.94%),49 (3.34%),2354 (97.96%),49 (2.04%)
9,P61_Juxta,1921,875,506 (57.83%),77 (7.36%),506 (86.79%),77 (13.21%)


In [64]:
import pandas as pd
import glob

csv_files = glob.glob('/SGRNJ06/randd/USER/zhouyiqi/work/analysis/20250721-vdj-crosstalk/public-10x/*_filtered_contig_annotations.csv')

summary_list = []

for csv_file in csv_files:
    sample = csv_file.split('/')[-1].split('_')[1] + '_' + csv_file.split('/')[-1].split('_')[2]
    if sample == 'P57_Blood':
        continue

    tcr_barcodes = pd.read_csv(csv_file).barcode.drop_duplicates()
    tcr_set = set(tcr_barcodes)

    

    # 获取当前 sample 的数据并去掉 doublet
    p = adata.obs[adata.obs['Sample ID'] == sample].copy()

    # 统计 doublet
    doublet_count = (p['annot_full'] == 'Doublet').sum()
    total_cells = len(p)
    doublet_pct = doublet_count / total_cells * 100 if total_cells > 0 else 0

    p = p[p['annot_full'] != 'Doublet']  # 去除 doublets
    p['barcode'] = p.index.str.split('_').str[-1]

    total_t_cells = sum(p['annot_full'] == 'T cells')
    total_non_t_cells = total_cells - total_t_cells - doublet_count

    p['has_TCR'] = p['barcode'].isin(tcr_set)
    p_has_tcr = p[p['has_TCR']].copy()
    p_has_tcr['is_T_cell'] = p_has_tcr['annot_full'] == 'T cells'

    count_series = p_has_tcr['is_T_cell'].value_counts()
    tcr_in_t_cells = count_series.get(True, 0)
    tcr_in_non_t_cells = count_series.get(False, 0)

    total_tcr_cells = tcr_in_t_cells + tcr_in_non_t_cells
    if total_tcr_cells == 0:
        tcr_in_t_cells_pct = 0
        tcr_in_non_t_cells_pct = 0
    else:
        tcr_in_t_cells_pct = tcr_in_t_cells / total_tcr_cells * 100
        tcr_in_non_t_cells_pct = tcr_in_non_t_cells / total_tcr_cells * 100

    t_cell_with_tcr_pct = (tcr_in_t_cells / total_t_cells * 100) if total_t_cells > 0 else 0
    non_t_cell_with_tcr_pct = (tcr_in_non_t_cells / total_non_t_cells * 100) if total_non_t_cells > 0 else 0

    def format_out(count, pct):
        return f"{count} ({pct:.2f}%)"

    summary_list.append({
        'Sample': sample,
        'Total Cells': total_cells,
        'Doublets': format_out(doublet_count, doublet_pct),
        'Total T Cells': total_t_cells,
        'T cell with TCR': format_out(tcr_in_t_cells, t_cell_with_tcr_pct),
        'Non-T cell with TCR': format_out(tcr_in_non_t_cells, non_t_cell_with_tcr_pct),
        #'TCR in T cells': format_out(tcr_in_t_cells, tcr_in_t_cells_pct),
        #'TCR in non-T cells': format_out(tcr_in_non_t_cells, tcr_in_non_t_cells_pct)
    })

summary_df = pd.DataFrame(summary_list)
summary_df = summary_df.sort_values('Sample')
summary_df.index.name = 'index'
summary_df


,Sample,Total Cells,Doublets,Total T Cells,T cell with TCR,Non-T cell with TCR
index,,,,,,
0,P47_Tumor,4752,20 (0.42%),4227,3171 (75.02%),32 (6.34%)
6,P55_Tumor,2190,1 (0.05%),2137,1474 (68.98%),0 (0.00%)
10,P57_Tumor,3850,8 (0.21%),1925,1701 (88.36%),27 (1.41%)
4,P58_Blood,10500,114 (1.09%),10341,6782 (65.58%),2 (4.44%)
9,P58_Tumor,1690,71 (4.20%),1535,834 (54.33%),1 (1.19%)
3,P60_Blood,10661,157 (1.47%),10398,10071 (96.86%),21 (19.81%)
2,P60_Juxta,2315,7 (0.30%),1917,1451 (75.69%),19 (4.86%)
1,P60_Tumor,10366,32 (0.31%),9678,8200 (84.73%),73 (11.13%)
5,P61_Blood,5749,24 (0.42%),5049,3213 (63.64%),6 (0.89%)
